In [35]:
!pip install tokenizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
!git clone https://github.com/cgjeong23/Viral-genomic-classification.git

Cloning into 'Viral-genomic-classification'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 50 (delta 17), reused 47 (delta 14), pack-reused 0
Unpacking objects: 100% (50/50), done.


In [ ]:
!mv Viral-genomic-classification virus

## Imports

In [41]:
from virus.ML.model import RnnModel, RnnModelForClassification
from virus.ML.train import train, evaluate
from virus.ML.dataloader import load_sequences, sample_data, get_3_splits, SequenceDataset
from torch import nn
from torch.utils.data import DataLoader

%load_ext autoreload
%autoreload 2

# Do Training

In [ ]:
base_path = '/content/drive/MyDrive/trainingdata'

tokenizer_file = '/content/drive/MyDrive/gene_tokenizer.json'

sequences, labels = load_sequences(base_path)

(train_seq, valid_seq, test_seq,
 train_label, valid_label, test_label) = get_3_splits(sequences, labels)

train_dataset = SequenceDataset(train_seq, train_label, tokenizer_file=tokenizer_file)
valid_dataset = SequenceDataset(valid_seq, valid_label, tokenizer_file=tokenizer_file,
                                label_dict=train_dataset.label_dict)
test_dataset = SequenceDataset(test_seq, test_label, tokenizer_file=tokenizer_file,
                               label_dict=train_dataset.label_dict)

In [ ]:
lr = 1e-4
batch_size = 5000
num_epochs = 1
vocab_size = train_dataset.tokenizer.get_vocab_size()
pad_id = train_dataset.tokenizer.padding['pad_id']
embedding_dim = 256
hidden_dim = 512
num_layers = 1

In [ ]:
#model = RnnModel(vocab_size, embedding_dim, pad_id, hidden_dim, num_layers)
model = RnnModelForClassification(vocab_size, embedding_dim, pad_id, hidden_dim, num_layers, len(train_dataset.label_dict))
model = model.to('cuda')
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=train_dataset.collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, collate_fn=valid_dataset.collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=test_dataset.collate_fn)
loss_function = nn.CrossEntropyLoss()

In [ ]:
acc_history = train(model, train_loader, loss_function, lr, num_epochs, 
                    valid_loader=valid_loader, test_loader=test_loader)

In [ ]:
acc_history

In [ ]:
import matplotlib.pyplot as plt

plt.plot(acc_history)